In [0]:
%run "../General/NB_Configuration"

In [0]:
spark.sql(f"""use catalog {catalog}""")

In [0]:
basetable = spark.sql('''
SELECT master.* FROM silver.customer_master master
INNER JOIN (
    SELECT a.SF_Key,
    CASE WHEN (a.Open) + (a.Pending) + (a.Dormant) > 0 THEN 'Active Accounts' ELSE 'Inactive Accounts' END AS ActiveAccountStatus
    FROM (
        SELECT ACCT.SF_CustKey_fkey AS SF_Key,
        SUM(acct.Closed) AS Closed,
        SUM(acct.Purged) AS Purged,
        SUM(acct.Open) AS Open,
        SUM(acct.Dormant) AS Dormant,
        SUM(acct.Pending) AS Pending,
        SUM(acct.Other) AS Other,
        SUM(acct.Missing) AS Missing,
        SUM(acct.Total) AS Total,
        SUM(acct.Closed) + SUM(acct.Purged) + SUM(acct.Open) + SUM(acct.Dormant) + SUM(acct.Pending) + SUM(acct.Other) + SUM(acct.Missing) AS Check
        FROM (
            SELECT CASE WHEN fa.status = 'CLOSED' and fa.ClosingDate < '2023-01-01' THEN 1 ELSE 0 END AS Closed,
            CASE WHEN fa.status = 'PURGED' THEN 1 ELSE 0 END AS Purged,
            CASE WHEN fa.status IN('OPEN') THEN 1
                WHEN fa.status IN('CLOSED') AND fa.ClosingDate >= '2023-01-01' THEN 1 ELSE 0 END AS Open,
            CASE WHEN fa.status = 'DORMANT' THEN 1 ELSE 0 END AS Dormant,
            CASE WHEN fa.status = 'PENDING' THEN 1 ELSE 0 END AS Pending,
            CASE WHEN fa.status = ' ' OR fa.status IS NULL THEN 1 ELSE 0 END AS Missing,
            CASE WHEN fa.status NOT IN ('OPEN', 'CLOSED', 'PURGED', 'DORMANT', 'PENDING') THEN 1 ELSE 0 END AS Other,
            1 AS Total,
            fap.SF_CustKey_fkey
            FROM silver.financial_account fa
            INNER JOIN silver.financial_account_party fap
            ON fa.financialaccountnumber = fap.Financial_AccountNumber_skey AND UPPER(fap.CurrentRecord) = 'YES'
            WHERE UPPER(fa.CurrentRecord) = 'YES'
        ) acct
        GROUP BY acct.SF_CustKey_fkey
    )a
)b ON b.sf_key = master.SF_Cust_Key
-- WHERE master.CurrentRecord = 'Yes'
WHERE ActiveAccountStatus = 'Active Accounts'
-- and b.ActiveAccountStatus = 'Active Accounts' 
-- removed filter for closed accounts since Erick requested on 6/17

''').drop('SF_Key').drop('ActiveAccountStatus')

basetable.createOrReplaceTempView("final_Account_vw")
basetable.count()

In [0]:
%sql
select sf_cust_key from final_Account_vw
group by 1 
having count(*) > 1
;

In [0]:
query = ''' SELECT * from final_Account_vw '''
df = spark.sql(query)
df.groupBy(['DW_modified_Date', 'currentRecord']).count().display()


In [0]:
basetable = spark.sql('''

-- Retrieve all financial accounts in ANY status, if they are tied to a customer with at least one "Active" financial account
-- Active: STATUS is Open, Pending, Dormant; Or Closed after 2023-01-01
SELECT * FROM (
SELECT *, row_number() OVER (PARTITION BY financialaccountnumber, currentRecord, DW_Modified_Date ORDER BY FinancialAccountNumber DESC) AS rn
FROM silver.financial_account fa WHERE financialaccountnumber IN
(
 -- Retrieve all financial account numbers tied to customer records with "Active Accounts"
 SELECT financial_accountnumber_skey FROM silver.financial_account_party FAP
 INNER JOIN
 (
  SELECT * FROM silver.customer_master master
  INNER JOIN 
  -- Filter customer records to only include those in the list with "Active Accounts"
  (
      SELECT a.SF_Key,
      CASE WHEN (a.Open) + (a.Pending) + (a.Dormant) > 0 THEN 'Active Accounts' ELSE 'Inactive Accounts' END AS ActiveAccountStatus
      FROM (
          SELECT ACCT.SF_CustKey_fkey AS SF_Key,
          SUM(acct.Closed) AS Closed,
          SUM(acct.Purged) AS Purged,
          SUM(acct.Open) AS Open,
          SUM(acct.Dormant) AS Dormant,
          SUM(acct.Pending) AS Pending,
          SUM(acct.Other) AS Other,
          SUM(acct.Missing) AS Missing,
          SUM(acct.Total) AS Total,
          SUM(acct.Closed) + SUM(acct.Purged) + SUM(acct.Open) + SUM(acct.Dormant) + SUM(acct.Pending) + SUM(acct.Other) + SUM(acct.Missing) AS Check
          FROM (
              SELECT CASE WHEN fa.status = 'CLOSED' and fa.ClosingDate < '2023-01-01' THEN 1 ELSE 0 END AS Closed,
              CASE WHEN fa.status = 'PURGED' THEN 1 ELSE 0 END AS Purged,
              CASE WHEN fa.status IN('OPEN') THEN 1
                  WHEN fa.status IN('CLOSED') AND fa.ClosingDate >= '2023-01-01' THEN 1 ELSE 0 END AS Open,
              CASE WHEN fa.status = 'DORMANT' THEN 1 ELSE 0 END AS Dormant,
              CASE WHEN fa.status = 'PENDING' THEN 1 ELSE 0 END AS Pending,
              CASE WHEN fa.status = ' ' OR fa.status IS NULL THEN 1 ELSE 0 END AS Missing,
              CASE WHEN fa.status NOT IN ('OPEN', 'CLOSED', 'PURGED', 'DORMANT', 'PENDING') THEN 1 ELSE 0 END AS Other,
              1 AS Total,
              fap.SF_CustKey_fkey
              FROM silver.financial_account fa
              INNER JOIN silver.financial_account_party fap
              ON fa.financialaccountnumber = fap.Financial_AccountNumber_skey AND UPPER(fap.CurrentRecord) = 'YES'
              WHERE UPPER(fa.CurrentRecord) = 'YES'
          ) acct
          GROUP BY acct.SF_CustKey_fkey
      )a
  )b ON b.sf_key = master.SF_Cust_Key
  WHERE master.CurrentRecord = 'Yes'
  AND ActiveAccountStatus = 'Active Accounts'
 ) CRM_ACCOUNT
 ON CRM_ACCOUNT.SF_CUST_KEY = FAP.SF_CUSTKEY_FKEY
 WHERE FAP.CURRENTRECORD = 'Yes'
 GROUP BY FINANCIAL_ACCOUNTNUMBER_SKEY
)
--AND FA.CURRENTRECORD = 'Yes'
)
--WHere rn =1
''').drop('rn').dropDuplicates(['financialaccountnumber', 'CurrentRecord', 'DW_Modified_Date'])
basetable.createOrReplaceTempView("final_FA_acct_vw")
print(basetable.count())
basetable.groupBy(['currentRecord', 'DW_Modified_Date']).count().display()

In [0]:
query = ''' Select * from silver.financial_account_balance''' 
df = spark.sql(query)
df.groupBy(['currentRecord', 'DW_Modified_Date']).count().display()

In [0]:
%sql
select FinancialAccountNumber, DW_Modified_Date, count(*) 
from final_FA_acct_vw
group by FinancialAccountNumber, currentrecord, DW_Modified_Date
having count(*) > 1
    

In [0]:
basetable = spark.sql('''
SELECT party.*
FROM silver.financial_account_party party
WHERE
party.Financial_AccountNumber_skey IN (SELECT distinct(FinancialAccountNumber) FROM final_FA_acct_vw)
--AND
--party.CurrentRecord = 'Yes'
''')
basetable.createOrReplaceTempView("final_fa_party_vw1")
print (basetable.count())

In [0]:
basetable = spark.sql('''
SELECT party.*
FROM final_fa_party_vw1 party
WHERE
party.sf_custkey_fkey IN(SELECT distinct(SF_Cust_Key) FROM final_Account_vw)
--AND
--party.CurrentRecord = 'Yes'
''').dropDuplicates(['Financial_AccountNumber_skey', 'SF_CustKey_fkey', 'DW_Modified_Date','currentrecord'])
basetable.createOrReplaceTempView("final_fa_party_vw")
print (basetable.count())

In [0]:
%sql
select  * from final_fa_party_vw WHERE SF_CustKey_fkey in (Select SF_Cust_Key FROM final_Account_vw)

In [0]:
%sql
Select Financial_AccountNumber_skey, SF_CustKey_fkey, currentrecord, dw_modified_date,count(*) from final_fa_party_vw
group by 1, 2, 3,4
having count(*)> 1

In [0]:
%sql
select count(*) from final_FA_acct_vw

In [0]:
%sql
SELECT financialaccountnumber, currentrecord FROM final_FA_acct_vw
GROUP BY 1,2
HAVING COUNT(*) > 1

In [0]:
# %sql
# drop table crm.Account;
# spark.sql('''DROP TABLE crm.financial_account''')
# drop table crm.financial_account_party;

basetable = spark.sql('''SELECT * FROM final_Account_vw''')
basetable.write.mode('overwrite').option('mergeSchema', 'true').saveAsTable('crm.Account')

basetable = spark.sql('''SELECT * FROM final_fa_party_vw''')
basetable.write.mode('overwrite').option('mergeSchema', 'true').saveAsTable('crm.financial_account_party')

basetable = spark.sql('''SELECT * FROM final_FA_acct_vw''')
basetable.write.mode('overwrite').option('mergeSchema', 'true').option('overwriteSchema', 'true').saveAsTable('crm.financial_account')

#Integrity Check

In [0]:
basetable = spark.sql('''
select * from CRM.account where CurrentRecord = 'Yes'
and SF_Cust_Key in(Select SF_CustKey_fkey from CRM.financial_account_party where CurrentRecord = 'Yes')
''')
basetable.createOrReplaceTempView("CRM_final_Account_vw")

In [0]:
basetable = spark.sql('''
SELECT party.*
FROM CRM.financial_account_party party
WHERE party.Financial_AccountNumber_skey in (Select FinancialAccountNumber FROM CRM.financial_account where CurrentRecord = 'Yes')
AND party.CurrentRecord = 'Yes'
''')
basetable.createOrReplaceTempView("CRM_final_fa_party_vw1")

In [0]:
basetable = spark.sql('''
SELECT party.*
FROM CRM_final_fa_party_vw1 party
WHERE party.sf_custkey_fkey in (Select SF_Cust_Key FROM CRM_final_Account_vw where CurrentRecord = 'Yes')
AND party.CurrentRecord = 'Yes'
''')
basetable.createOrReplaceTempView("crm_final_fa_party_vw")

In [0]:
basetable = spark.sql('''
SELECT acct.*
FROM crm.financial_account acct
    WHERE acct.FinancialAccountNumber not in (SELECT Financial_AccountNumber_skey from crm_final_fa_party_vw where acct.currentrecord = 'Yes')
AND acct.CurrentRecord = 'Yes'
AND acct.Status in ('OPEN', 'DORMANT', 'PENDING')
''')
basetable.createOrReplaceTempView("crm_final_FA_acct_vw")

In [0]:
# if basetable.count()>0:
#     raise "Integrity Check Failed"